The database operates on ingredient IDs. It consists of a list of fresh ingredient ID ranges, a blank line, and a list of available ingredient IDs. For example:
```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```

The fresh ID ranges are inclusive: the range 3-5 means that ingredient IDs 3, 4, and 5 are all fresh. The ranges can also overlap; an ingredient ID is fresh if it is in any range.

## Part 1

In [1]:
data_test = """3-5
10-14
16-20
12-18

1
5
8
11
17
32"""

In [2]:
# from itertools import chain
# Tooo slow
# def solve_part1(data):
#     i_fresh, i_av = data.split("\n\n")
#     i_fresh = list(
#         chain.from_iterable(
#             [
#                 range(
#                     int(i_range.split("-")[0]), 
#                     int(i_range.split("-")[1]) + 1
#                 )
#                 for i_range in i_fresh.splitlines()
#             ]
#         )
#     )
#     i_fresh
#     i_av = [int(i) for i in i_av.splitlines()]
#     i_av_fresh = [i for i in i_av if i in i_fresh]
#     return len(i_av_fresh)

In [2]:
from aocd import get_data
data = get_data(day=5, year=2025)

In [3]:
# faster
def solve_part1(data):
    i_fresh, i_av = data.split("\n\n")
    i_fresh = [
                range(
                    int(i_range.split("-")[0]), 
                    int(i_range.split("-")[1]) + 1
                )
                for i_range in i_fresh.splitlines()
            ]
    i_av = [int(i) for i in i_av.splitlines()]
    i_av_fresh = set([i for i in i_av for fresh_range in i_fresh if i in fresh_range])
    return len(i_av_fresh)

In [4]:
solve_part1(data_test)

3

In [5]:
solve_part1(data)

744

## Part 2

In [6]:
def size(r):
    s,e = r
    return e-s+1

def first_overlap(r1, ranges):
    for r2 in ranges:
        if overlaps(r1, r2) > 0:
            return r2
    return None

# first_overlap((10,15), i_fresh[2:])

In [7]:
def overlaps(r1, r2):
    s1, e1 = r1
    s2, e2 = r2
    if r1 == r2:
        # print("same!")
        return size(r1)
    elif s1<=s2 and e1>=e2:
        # print("complete OL: r2 inside r1", r1, r2)
        return size(r2)
    elif s1>=s2 and e1<=e2:
        # print("complete OL: r1 inside r2", r1, r2)
        return size(r2)
    elif s1<s2 and e1<e2 and e1>=s2:
        # print("OL right", r1, r2)
        return e1-s2+1
    elif s1>s2 and e1>e2 and e2>=s1:
        # print("OL LEFT", r1, r2)
        return e2-s1+1
    return 0


In [8]:
def fresh_ranges(data):
    i_fresh, i_av = data.split("\n\n")
    i_fresh = [
        (int(i_range.split("-")[0]), int(i_range.split("-")[1]))
        for i_range in i_fresh.splitlines()
    ]
    return i_fresh
fresh_ranges(data_test)

[(3, 5), (10, 14), (16, 20), (12, 18)]

In [9]:
def solve_part2(data, max_iters=10):
    uniq_ranges = fresh_ranges(data)
    
    iter_num, found_overlap = 0, True
    while found_overlap:
        found_overlap = False
        seen = set()
        _uniq_ranges = []
        iter_num+=1
        # print(f"** Iteration {iter_num}")
        # print(uniq_ranges)
        for idx in range(len(uniq_ranges)):
            r = uniq_ranges[idx]
            if r in seen: continue
            # print(" Testing", r, uniq_ranges[idx+1:])
            r_ol = first_overlap(r, uniq_ranges[idx+1:])
            if r_ol and not r_ol in seen:
                r_merged = (min(r[0], r_ol[0]), max(r[1], r_ol[1]))
                # print("Adding merged", r_merged)
                seen.add(r)
                seen.add(r_ol)
                found_overlap = True
                _uniq_ranges.append(r_merged)
            else:
                # print("Adding solo", r)
                _uniq_ranges.append(r)
                seen.add(r)   
        uniq_ranges = _uniq_ranges
        if iter_num>max_iters:
            break
    return sum([size(r) for r in uniq_ranges])

In [11]:
solve_part2(data_test)

14

In [12]:
solve_part2(data)

347468726696961